In [1]:
import torch
import gpytorch as gp
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score

torch.manual_seed(1)
np.random.seed(1)

In [2]:
df = pd.read_csv('C:/Users/tln229/Downloads/Python/1. Building/data/HVAC_B90_102_exp_10m_20210424.csv')

In [3]:
class MyGPModel(gp.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(MyGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module  = gp.means.ZeroMean()
        self.covar_module = gp.kernels.ScaleKernel(gp.kernels.RBFKernel())

    def forward(self, x):
        mean_x  = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gp.distributions.MultivariateNormal(mean_x, covar_x)

In [4]:
for n_train in ([32, 64, 128]):
    for lr in ([0.001, 0.01, 0.02, 0.05]):

        # TRAIN SET
        clg_sp       = np.array(df['clg_sp_current']).reshape(-1,1)[0: n_train]
        htg_sp       = np.array(df['htg_sp_current']).reshape(-1,1)[0: n_train]
        htg_clg_mode = 1*np.array(df['htg_clg_mode']).reshape(-1,1)[0: n_train]

        sp_k   = htg_sp*htg_clg_mode + clg_sp*(1-htg_clg_mode)
        Tz_k   = np.array(df['thermostat_room_temp']).reshape(-1,1)[0: n_train]
        qrh_k  = np.array(df['htg_valve_command']).reshape(-1,1)[0: n_train]
        qrh_k1 = np.array(df['htg_valve_command'])[0+1: n_train+1]

        train_x  = np.concatenate((sp_k, Tz_k, qrh_k), axis=1)
        train_x = torch.tensor(train_x, dtype=torch.float32)
        train_y = torch.tensor(qrh_k1, dtype=torch.float32)

        # TEST SET
        clg_sp       = np.array(df['clg_sp_current']).reshape(-1,1)[n_train: 1600]
        htg_sp       = np.array(df['htg_sp_current']).reshape(-1,1)[n_train: 1600]
        htg_clg_mode = 1*np.array(df['htg_clg_mode']).reshape(-1,1)[n_train: 1600]

        sp_k   = htg_sp*htg_clg_mode + clg_sp*(1-htg_clg_mode)
        Tz_k   = np.array(df['thermostat_room_temp']).reshape(-1,1)[n_train: 1600]
        qrh_k  = np.array(df['htg_valve_command']).reshape(-1,1)[n_train: 1600]
        qrh_k1 = np.array(df['htg_valve_command'])[n_train+1: 1600+1]
        msa_k1 = np.array(df['airflow_desired'])[n_train+1: 1600+1]

        test_x  = np.concatenate((sp_k, Tz_k, qrh_k), axis=1)
        test_x  = torch.tensor(test_x, dtype=torch.float32)
        test_y  = torch.tensor(qrh_k1, dtype=torch.float32)
        test_y2 = torch.tensor(msa_k1, dtype=torch.float32)

        # initialize likelihood and model
        likelihood = gp.likelihoods.GaussianLikelihood()
        model      = MyGPModel(train_x, train_y, likelihood)

        # Training
        training_iter = 200000    # number of training iteration

        optimizer = torch.optim.Adam(model.parameters(), lr=lr)       # optimizer
        mll = gp.mlls.ExactMarginalLogLikelihood(likelihood, model)   # marginal likelihood

        R2_qrh = np.array([], dtype=np.float32)
        R2_msa = np.array([], dtype=np.float32)
        for i in range(training_iter+1):
            # TRAIN
            model.train()                   # find the hyperparameters
            likelihood.train()

            optimizer.zero_grad()           # Zero gradients from previous iteration
            output = model(train_x)         # Output from model
            loss   = -mll(output, train_y)  # Calc loss and backprop gradients

            loss.backward()
            optimizer.step()
            
            # EVAL
            model.eval()
            likelihood.eval()

            with torch.no_grad(), gp.settings.fast_pred_var():
                pred_y = likelihood(model(test_x))

            # r2 qrh
            r2 = r2_score(test_y.numpy(), pred_y.mean.numpy())
            R2_qrh = np.append(R2_qrh, r2)
            
            # r2 msa
            qrh = np.array(df['htg_valve_position']).reshape(-1,1)
            msa = np.array(df['airflow_current']).reshape(-1,1)

                # LEAST SQUARE
            ones = np.ones(msa.shape)
            A = np.concatenate((qrh, ones), axis=1)
            b = np.copy(msa)
            p = np.linalg.lstsq(A, b, rcond=None)[0]
            pred_msa = pred_y.mean.numpy()*p[0] + p[1]
            r2_msa = r2_score(test_y2.numpy(), pred_msa)
            R2_msa = np.append(R2_msa, r2_msa)
        
        # print(np.argmax(R2_test))
        print('n:%4d \t lr:%5.3f \t r2_qrh:%8.4f \t r2_msa:%8.4f' % (n_train, lr, np.max(R2_qrh), np.max(R2_msa)))

RuntimeError: Can only multiply by scalars